In [2]:
# credit to https://www.kaggle.com/mtszkw/xgboost-for-stock-trend-prices-prediction for XGBoost documentation 
import pyEX as p
import pandas as pd
import seaborn as sns
import numpy as np
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

init_notebook_mode(connected=True)

#Input data and create a weighted average of bid and ask prices
df = pd.read_csv('processedlevel2data.csv')
df['average'] = df[['bidprice', 'askprice']].mean(axis=1)
df['average'] = np.where(df['average'] < 150, df[['bidprice', 'askprice']].max(axis=1), df['average'])
df['time'] = df['asktimestamp'].copy()
df.time.fillna(df.bidtimestamp, inplace=True)
df = df[df.average != 0]

In /Users/benjaminliu/opt/anaconda3/envs/myenv/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/benjaminliu/opt/anaconda3/envs/myenv/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/benjaminliu/opt/anaconda3/envs/myenv/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/benjaminliu/opt/anaconda3/envs/myenv/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

In [3]:
# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(103, 128, 159, .8)')
fig = go.Figure(layout=layout)
fig.update_layout(
    font_color="white",
)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

In [4]:
df = df.iloc[33:] # Because of moving averages and MACD line
df = df[:-1]      # Because of shifting close price

df.index = range(len(df))

test_size  = 0.15
valid_size = 0.15

test_split_idx  = int(df.shape[0] * (1-test_size))
valid_split_idx = int(df.shape[0] * (1-(valid_size+test_size)))

train_df  = df.loc[:valid_split_idx].copy()
valid_df  = df.loc[valid_split_idx+1:test_split_idx].copy()
test_df   = df.loc[test_split_idx+1:].copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=train_df.time, y=train_df.average, name='Training'))
fig.add_trace(go.Scatter(x=valid_df.time, y=valid_df.average, name='Validation'))
fig.add_trace(go.Scatter(x=test_df.time,  y=test_df.average,  name='Test'))
fig.show()

drop_cols = ['symbol', 'messageType', 'bidtimestamp', 'asktimestamp', 'time']

train_df = train_df.drop(drop_cols, 1)
valid_df = valid_df.drop(drop_cols, 1)
test_df  = test_df.drop(drop_cols, 1)
print(train_df)

     index   seq  bidprice  bidsize  askprice  asksize  average
0       71  1852    196.93    100.0    197.25    100.0   197.09
1       72  1853      0.00      0.0    197.25    100.0   197.25
2       73  1854    196.93    100.0    197.25    100.0   197.09
3       75  1855    196.87    100.0    197.25    100.0   197.06
4       76  1856      0.00      0.0    197.25    100.0   197.25
..     ...   ...       ...      ...       ...      ...      ...
280    493  2235    195.94    100.0    196.28    100.0   196.11
281    494  2236      0.00      0.0    196.22    100.0   196.22
282    495  2237      0.00      0.0    196.22    100.0   196.22
283    496  2238      0.00      0.0    196.22    100.0   196.22
284    498  2239      0.00      0.0    196.27    100.0   196.27

[285 rows x 7 columns]


In [5]:
y_train = train_df['average'].copy()
X_train = train_df.drop(['average'], 1)

y_valid = valid_df['average'].copy()
X_valid = valid_df.drop(['average'], 1)

y_test  = test_df['average'].copy()
X_test  = test_df.drop(['average'], 1)

X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     285 non-null    int64  
 1   seq       285 non-null    int64  
 2   bidprice  285 non-null    float64
 3   bidsize   285 non-null    float64
 4   askprice  285 non-null    float64
 5   asksize   285 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 13.5 KB


In [13]:
parameters = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_depth': [8, 10, 12, 15],
    'gamma': [0.001, 0.005, 0.01, 0.02],
    'random_state': [42]
}

eval_set = [(X_train, y_train), (X_valid, y_valid)]
model = xgb.XGBRegressor(eval_set=eval_set, objective='reg:squarederror', verbose = False)

model = xgb.XGBRegressor(eval_set = [(X_train, y_train), (X_valid, y_valid)], objective='reg:squarederror')
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

[15:45:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { eval_set } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1,
             eval_set=[(     index   seq  bidprice  bidsize  askprice  asksize
0       71  1852    196.93    100.0    197.25    100.0
1       72  1853      0.00      0.0    197.25    100.0
2       73  1854    196.93    100.0    197.25    100.0
3       75  1855    196.87    100.0    197.25    100.0
4       76  1856      0.00      0.0    197.25    100.0
..     ...   ...       ...      ...       ...      ...
280    493  2235    195.94    100.0    196.28    100.0
281    494  223...
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tre

In [14]:
y_pred = model.predict(X_test)
print(f'y_true = {np.array(y_test)[:5]}')
print(f'y_pred = {y_pred[:5]}')

y_true = [196.77 196.82 196.75 197.11 197.11]
y_pred = [196.26044 196.28038 196.25824 196.8058  196.8058 ]


In [15]:
predicted_prices = df.loc[test_split_idx+1:].copy()
predicted_prices['Close'] = y_pred

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=df.time, y=df.average,
                         name='Truth',
                         marker_color='LightSkyBlue'), row=1, col=1)

fig.add_trace(go.Scatter(x=predicted_prices.time,
                         y=predicted_prices.average,
                         name='Prediction',
                         marker_color='MediumPurple'), row=1, col=1)

fig.add_trace(go.Scatter(x=predicted_prices.time,
                         y=y_test,
                         name='Truth',
                         marker_color='LightSkyBlue',
                         showlegend=False), row=2, col=1)

fig.add_trace(go.Scatter(x=predicted_prices.time,
                         y=y_pred,
                         name='Prediction',
                         marker_color='MediumPurple',
                         showlegend=False), row=2, col=1)

fig.show()